In [33]:
import ForwardDiff as fd
import Zygote as zg
import LinearAlgebra as la

5-element Vector{Float64}:
 0.638899529814582
 0.7582517136440069
 0.01708976447027355
 0.1875862086154486
 0.14750488148977603

In [107]:
function sig(x)
    return 1 ./ (1 .+ exp.(-x))
end

sig (generic function with 1 method)

In [108]:
function sig_der!(delta, output)
    delta .= delta.* output .* (1 .- output)
end

sig_der! (generic function with 1 method)

In [148]:
x = rand(5)

W1 = rand(5, 5)
b1 = rand(5)

W2 = rand(2, 5)
b2 = rand(2)

W3 = rand(3, 2)
b3 = rand(3)

linear(x) = sum(sig(W3 * sig(W2 * sig(W1 * x + b1) .+ b2) .+ b3))

grads = zg.gradient(() -> sum(linear(x)), zg.Params([W2, b2, W1, b1, W3, b3]))

Grads(...)

In [149]:
delta = ones(3)

sig_der!(delta, sig(W3 * sig(W2 * sig(W1 * x + b1) .+ b2) .+ b3))

print(delta * transpose(sig(W2 * sig(W1 * x + b1) .+ b2)) ≈ grads[W3])
print(delta ≈ grads[b3])
delta = transpose(W3) * delta

sig_der!(delta, sig(W2 * sig(W1 * x + b1) .+ b2))

print(delta * transpose(sig(W1 * x + b1)) ≈ grads[W2])
print(delta ≈ grads[b2])
delta = transpose(W2) * delta

sig_der!(delta, sig(W1 * x + b1))

print(delta * transpose(x) ≈ grads[W1])
print(delta ≈ grads[b1])

truetruetruetruetruetrue

In [ ]:
1x5

## Kinetic

In [160]:
linear(x) = sum(sig(W1 * x + b1))

hess = la.tr(zg.hessian(linear, x))

-0.4607676558130107